In [2]:
# 기본 패키지
import os
import re
import json
import copy
import warnings
import platform
from collections import Counter

# 데이터 분석
import pandas as pd
import numpy as np

# 형태소 분석
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords

# 경고 무시 설정
warnings.filterwarnings('ignore')

In [3]:
# Kiwi 형태소 분석기 및 불용어 설정
kiwi = Kiwi()
stopwords = Stopwords()

# 감성 사전 로드
with open('data/SentiWord_info.json', encoding='utf-8-sig') as f:
    sentiword = json.load(f)
senti_dict = {entry['word']: int(entry['polarity']) for entry in sentiword}

# 분석에 사용할 품사 태그
useful_tags = {'NNG', 'NNP', 'VV', 'VA', 'VX', 'VCP', 'VCN', 'MAG', 'MAJ', 'XR', 'IC'}

In [4]:
# 감성 점수를 감성 레이블(긍정/부정/중립)로 변환
def get_sentiment_label(sentence):
    tokens = kiwi.tokenize(sentence, normalize_coda=True, stopwords=stopwords)
    filtered = [t.form for t in tokens if t.tag in useful_tags]
    total_score = sum(senti_dict.get(word, 0) for word in filtered)
    if total_score > 0:
        return '긍정'
    elif total_score < 0:
        return '부정'
    else:
        return '중립'

In [5]:
# 한글만 추출
def extract_korean(text):
    return ' '.join(re.findall(r'[가-힣]+', str(text))) if pd.notnull(text) else ''

In [6]:
# 문장 단위 분리 및 감성 분석
def extract_all_sentiment_sentences(df, date_col='trip_day'):
    sent_rows = []
    for _, row in df.iterrows():
        date = row.get(date_col, None)
        review = extract_korean(row['review'])
        sentences = kiwi.split_into_sents(review)
        for s in sentences:
            sent_text = s.text.strip()
            if sent_text:
                sentiment = get_sentiment_label(sent_text)
                sent_rows.append({
                    'date': date,
                    'sentence': sent_text,
                    'sentiment': sentiment
                })
    return pd.DataFrame(sent_rows)

In [7]:
# 데이터 폴더 및 파일 목록 설정
folder_path = './data'
filenames = [
    'Merged_ANA.csv',
    'Merged_JAL.csv',
    'Merged_PEACH.csv',
    'Merged_대한항공.csv',
    'Merged_아시아나.csv',
    'Merged_에어서울.csv',
    'Merged_제주항공.csv',
    'Merged_진에어.csv',
    'Merged_티웨이.csv'
]

In [8]:
summary_rows = []

# 각 항공사 데이터 처리
for fname in filenames:
    path = os.path.join(folder_path, fname)
    try:
        df = pd.read_csv(path, encoding='utf-8')

        if 'review' not in df.columns:
            print(f"⚠️ '{fname}'에 'review' 열이 없습니다.")
            continue

        if 'trip_day' not in df.columns:
            df['trip_day'] = None

        airline_name = fname.replace('Merged_', '').replace('.csv', '')

        # 문장 단위 감성 분석
        sentiment_df = extract_all_sentiment_sentences(df)

        # 파일 저장
        output_path = f"results/sentiment_sentences_{airline_name}.csv"
        os.makedirs('results', exist_ok=True)
        sentiment_df.to_csv(output_path, index=False, encoding='utf-8-sig')

        # 감성 분포 요약
        counts = Counter(sentiment_df['sentiment'])
        total = sum(counts.values())
        summary_rows.append({
            '항공사': airline_name,
            '총 문장 수': total,
            '긍정(%)': round(counts['긍정'] / total * 100, 2) if total else 0,
            '중립(%)': round(counts['중립'] / total * 100, 2) if total else 0,
            '부정(%)': round(counts['부정'] / total * 100, 2) if total else 0
        })

        print(f"✅ {airline_name}: 분석 완료 → {output_path}")

    except Exception as e:
        print(f"❗ 파일 {fname} 처리 중 오류 발생: {e}")

✅ ANA: 분석 완료 → results/sentiment_sentences_ANA.csv
✅ JAL: 분석 완료 → results/sentiment_sentences_JAL.csv
✅ PEACH: 분석 완료 → results/sentiment_sentences_PEACH.csv
✅ 대한항공: 분석 완료 → results/sentiment_sentences_대한항공.csv
✅ 아시아나: 분석 완료 → results/sentiment_sentences_아시아나.csv
✅ 에어서울: 분석 완료 → results/sentiment_sentences_에어서울.csv
✅ 제주항공: 분석 완료 → results/sentiment_sentences_제주항공.csv
✅ 진에어: 분석 완료 → results/sentiment_sentences_진에어.csv
✅ 티웨이: 분석 완료 → results/sentiment_sentences_티웨이.csv


In [11]:
# 감성 비율 요약
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv("results/sentiment_summary_by_airline.csv", index=False, encoding='utf-8-sig')
display(summary_df)

,항공사,총 문장 수,긍정(%),중립(%),부정(%)
0,ANA,2322,28.64,51.34,20.03
1,JAL,2441,30.52,51.00,18.48
2,PEACH,952,21.32,48.11,30.57
3,대한항공,4881,28.70,52.51,18.79
4,아시아나,5581,27.56,52.52,19.92
5,에어서울,2723,21.70,52.26,26.04
6,제주항공,4663,20.93,51.86,27.21
7,진에어,1869,27.50,51.36,21.13
8,티웨이,1703,23.02,50.97,26.01
